# Thực hành phân loại văn bản

# 1. Nhập dataset và kiểm tra

In [1]:
import warnings
import pandas as pd

warnings.filterwarnings('ignore')
df = pd.read_csv('./data/data_sentiment.csv')
df.dropna(axis=0, inplace=True)

In [2]:
df.sample(5, random_state=0)

,comment,sentiment
78156,***UPDATE*****2020 APP IS STILL TERRIBLE!! HOW...,-1
534330,Its extremely hard to setup temp users. There ...,1
646491,Doesn't work at all,-1
603393,I really love this game,1
457729,great! my son enjoys it a lot 😊,1


In [3]:
df.groupby('sentiment').sentiment.count()

sentiment
-1    383707
 1    753150
Name: sentiment, dtype: int64

positive: 1 \
negative: -1

In [4]:
# Kiểm tra kích thước dataset
df.shape

(1136857, 2)

# 2. Làm sạch dữ liệu

1. Chuyển tất cả các từ sang chữ thường.
2. Loại bỏ tất cả các dấu chấm câu, số, từ ít hơn 3 ký tự.
3. Tách từ
4. Loại bỏ stopwords
5. Đưa về từ nguyên gốc

In [5]:
#Import các thư viện cần thiết
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer

STOPWORDS = set(stopwords.words('english'))

In [6]:
def preprocess(sentence):
    sentence = sentence.lower()
    stemmer = SnowballStemmer("english")
    tokenizer = RegexpTokenizer(r'[a-z]{3,}')
    tokens = tokenizer.tokenize(sentence)
    tokens = [stemmer.stem(w) for w in tokens if w not in STOPWORDS]

    return " ".join(tokens)

In [7]:
df['comment'] = df['comment'].apply(preprocess)
df['comment'].head()

0    gamer rememb fond fli game includ window game ...
1    good game play bore love anim game especi dest...
2    fun game lot custom beauti plane mani add watc...
3    love graphic awesom gameplay awesom challeng o...
4    tri least twenti differ game hng warplan best ...
Name: comment, dtype: object

# 3. Tạo mô hình Bag of Words sử dụng sklearn

Tách dataset thành tập train và test

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['comment'], df['sentiment'], test_size=.2, random_state=42)

print('Number of rows in the train set: {}'.format(X_train.shape[0]))
print('Number of rows in the test set: {}'.format(X_test.shape[0]))

Number of rows in the train set: 909485
Number of rows in the test set: 227372


In [9]:
from sklearn.feature_extraction.text import CountVectorizer

# Khởi tạo CountVectorizer method
count_vector = CountVectorizer()

# Fit và transform tập train rồi trả về ma trận
training_data = count_vector.fit_transform(X_train)

# Transform rồi trả về ma trận.
# Lưu ý không fit với CountVectorizer()
testing_data = count_vector.transform(X_test)

# 4. Huấn luyên mô hình sử dụng mô hình Naive Bayes

In [10]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB().fit(training_data, y_train)

In [11]:
test_predictions = model.predict(testing_data)

# 5. Đánh giá mô hình Naive Bayes

In [12]:
from sklearn.metrics import classification_report, confusion_matrix, f1_score

print('\nTest')
print(confusion_matrix(y_test, test_predictions))
print(classification_report(y_test, test_predictions))
print(f1_score(y_test, test_predictions))


Test
[[ 40126  36670]
 [ 11996 138580]]
              precision    recall  f1-score   support

          -1       0.77      0.52      0.62     76796
           1       0.79      0.92      0.85    150576

    accuracy                           0.79    227372
   macro avg       0.78      0.72      0.74    227372
weighted avg       0.78      0.79      0.77    227372

0.8506380706266534


# 6. Lưu mô hình

In [13]:
import pickle
import pandas as pd

In [14]:
with open('model', 'wb') as picklefile:
    pickle.dump(model, picklefile)
    
with open('vectorizer', 'wb') as picklefile:
    pickle.dump(count_vector, picklefile)

In [15]:
with open('model', 'rb') as training_model:
    model = pickle.load(training_model)
    
with open('vectorizer', 'rb') as vectorizer:
    count_vector = pickle.load(vectorizer)

In [16]:
data = {'comments':  ['love app']}

dfa = pd.DataFrame(data, columns=['comments'])

print(dfa)

   comments
0  love app


In [17]:
tmp = model.predict(count_vector.transform(dfa['comments']))

In [18]:
tmp

array([1])

In [19]:
tmp[0]

1